In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-point-forecast-valweights-actual/weights_validation.csv


In [2]:
from collections import defaultdict
import sys

In [3]:
sell_prices = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')

In [4]:
sell_prices = sell_prices.set_index(['item_id', 'store_id', 'wm_yr_wk'])

In [5]:
sell_prices

sell_price
item_id       store_id wm_yr_wk            
HOBBIES_1_001 CA_1     11325           9.58
                       11326           9.58
                       11327           8.26
                       11328           8.26
                       11329           8.26
...                                     ...
FOODS_3_827   WI_3     11617           1.00
                       11618           1.00
                       11619           1.00
                       11620           1.00
                       11621           1.00

[6841121 rows x 1 columns]

In [6]:
sell_prices.loc[('HOBBIES_1_001', 'CA_1', 11327)].values[0]

8.26

In [7]:
calendar = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv', parse_dates=['date'])

In [8]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [9]:
calendar = calendar.set_index('d')

In [10]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
d,,,,,,,,,,,,,
d_1,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
d_2,2011-01-30,11101,Sunday,2,1,2011,NaN,NaN,NaN,NaN,0,0,0
d_3,2011-01-31,11101,Monday,3,1,2011,NaN,NaN,NaN,NaN,0,0,0
d_4,2011-02-01,11101,Tuesday,4,2,2011,NaN,NaN,NaN,NaN,1,1,0
d_5,2011-02-02,11101,Wednesday,5,2,2011,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_1965,2016-06-15,11620,Wednesday,5,6,2016,NaN,NaN,NaN,NaN,0,1,1
d_1966,2016-06-16,11620,Thursday,6,6,2016,NaN,NaN,NaN,NaN,0,0,0
d_1967,2016-06-17,11620,Friday,7,6,2016,NaN,NaN,NaN,NaN,0,0,0


In [11]:
calendar.loc['d_1968', 'wm_yr_wk']

11621

In [12]:
sales_train_val = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')

In [13]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [14]:
def show_progress(progress_str, n_cols=80):
    sys.stdout.write(progress_str + (" " * max(0, n_cols - len(progress_str))) + "\r")
    sys.stdout.flush()

In [15]:
weights_df_dict = defaultdict(list)
for i in range(len(sales_train_val)):
    show_progress(f"Processing row {i+1}/{len(sales_train_val)}")
    _item_id = sales_train_val.item_id.iloc[i]
    _store_id = sales_train_val.store_id.iloc[i]
    _weight = 0.0
    _n = 0
    _ssum = 0.0
    _sales = []
    _day_of_first_sale = 1914
    
    for _d in range(1886,1914):
        _day_sales = sales_train_val['d_'+str(_d)].iloc[i]
        _wk_id = calendar.loc['d_'+str(_d), 'wm_yr_wk']
        _sp = sell_prices.loc[(_item_id, _store_id, _wk_id)].values[0]
        _weight += _day_sales * _sp
        
    for _d in range(1,1914):
        _day_sales = sales_train_val['d_'+str(_d)].iloc[i]
        if _n == 0:
            if _day_sales > 0:
                _n += 1
                _day_of_first_sale = _d
        else:
            _n += 1
        if _n>=2:
            _ssum += (_day_sales - sales_train_val['d_'+str(_d-1)].iloc[i])**2
            _sales.append(_day_sales)
    _sales = np.array(_sales).astype(np.float64)
    weights_df_dict['item_id'].append(_item_id)
    weights_df_dict['store_id'].append(_store_id)
    weights_df_dict['day_of_first_sale'].append(_day_of_first_sale)
    weights_df_dict['weight'].append(_weight)
    _all_zeros = 0
    if _n == 0:
        K = 0
        _all_zeros = 1
    else:
        K = pow(float(_n-1)/(_ssum), 0.5)
    weights_df_dict['K'].append(K)
    weights_df_dict['all_zeros'].append(_all_zeros)
    weights_df_dict['sales_mean'].append(_sales.mean())
    weights_df_dict['sales_std'].append(_sales.std())
weights_df_dict['weight'] = np.array(weights_df_dict['weight']).astype(np.float64)
weights_df_dict['K'] = np.array(weights_df_dict['K']).astype(np.float64)
weights_df_dict['all_zeros'] = np.array(weights_df_dict['all_zeros']).astype(np.int8)
weights_df_dict['sales_mean'] = np.array(weights_df_dict['sales_mean']).astype(np.float64)
weights_df_dict['sales_std'] = np.array(weights_df_dict['sales_std']).astype(np.float64)
weights_df_dict['day_of_first_sale'] = np.array(weights_df_dict['day_of_first_sale']).astype(np.int16)
weights_df = pd.DataFrame(weights_df_dict)

In [16]:
weights_df.loc[weights_df.all_zeros == 1]

,item_id,store_id,day_of_first_sale,weight,K,all_zeros,sales_mean,sales_std


In [17]:
wsum = weights_df.weight.sum(); wsum

3748180.05

In [18]:
weights_df['weight'] = weights_df.weight / wsum

In [19]:
weights_df

,item_id,store_id,day_of_first_sale,weight,K,all_zeros,sales_mean,sales_std
0,HOBBIES_1_001,CA_1,902,0.000060,0.855926,0,0.592483,0.849799
1,HOBBIES_1_002,CA_1,144,0.000002,1.278055,0,0.278123,0.586817
2,HOBBIES_1_003,CA_1,1106,0.000013,1.128216,0,0.353160,0.691916
3,HOBBIES_1_004,CA_1,37,0.000063,0.372943,0,1.751599,1.994428
4,HOBBIES_1_005,CA_1,113,0.000029,0.611990,0,1.022778,1.300441
...,...,...,...,...,...,...,...,...
30485,FOODS_3_823,WI_3,3,0.000005,0.863262,0,0.534031,1.177291
30486,FOODS_3_824,WI_3,6,0.000006,0.981145,0,0.374934,0.813791
30487,FOODS_3_825,WI_3,2,0.000027,0.659478,0,0.892726,1.380701
30488,FOODS_3_826,WI_3,940,0.000009,0.677787,0,0.720452,1.249856


In [20]:
weights_val_val = pd.read_csv('../input/m5-point-forecast-valweights-actual/weights_validation.csv')

In [21]:
weights_val = weights_val_val.set_index(['Agg_Level_1', 'Agg_Level_2'])

In [22]:
weights_val.loc[list(zip(weights_df.item_id.values, weights_df.store_id.values)), 'Weight']

Agg_Level_1    Agg_Level_2
HOBBIES_1_001  CA_1           0.000060
HOBBIES_1_002  CA_1           0.000002
HOBBIES_1_003  CA_1           0.000013
HOBBIES_1_004  CA_1           0.000063
HOBBIES_1_005  CA_1           0.000029
                                ...   
FOODS_3_823    WI_3           0.000005
FOODS_3_824    WI_3           0.000006
FOODS_3_825    WI_3           0.000027
FOODS_3_826    WI_3           0.000009
FOODS_3_827    WI_3           0.000008
Name: Weight, Length: 30490, dtype: float64

In [23]:
_a = weights_df.weight.values

In [24]:
_b = weights_val.loc[list(zip(weights_df.item_id.values, weights_df.store_id.values)), 'Weight'].values

In [25]:
(_a - _b).min(),(_a - _b).max() 

(-4.9978131386720434e-08, 4.9998888127051277e-08)

In [26]:
weights_df.to_pickle('weights_df')

In [27]:
!ls -alrh

total 1.7M
-rw-r--r-- 1 root root 1.7M Jun 20 13:32 weights_df
---------- 1 root root 7.6K Jun 20 12:51 __notebook__.ipynb
drwxr-xr-x 6 root root 4.0K Jun 20 12:51 ..
drwxr-xr-x 2 root root 4.0K Jun 20 13:32 .
